In [1]:
import random
from collections import Counter

In [2]:
# Define problem constraints
courses = ['C1', 'C2', 'C3', 'C4', 'C5']
exam_halls = ['H1', 'H2']
time_slots = ['T1', 'T2','T3']
max_hours = {'H1': 5, 'H2': 5}
conflicts = {('C1', 'C2'): 10, ('C1', 'C4'): 5, ('C2', 'C5'): 7, ('C3', 'C4'): 12, ('C4', 'C5'): 8}

In [3]:
# Since the solution is complex (i.e list of tuples) therefore this  
# helper function helps us in generating a solution
def create_solution():
    solution = []
    for course in courses:
        exam_time = random.choice(time_slots)
        exam_hall = random.choice(exam_halls)
        solution.append((course, exam_time, exam_hall))
    return solution

# Fitness function for the problem
def fitness_function(solution):
    fitness = 0
    for course1, time1, hall1 in solution:
        for course2, time2, hall2 in solution:
            if course1 != course2:
                if time1 == time2 and hall1 == hall2:
                    fitness += conflicts.get((course1, course2), 0)
    h_list = []
    
    # adding fitness on basis of the usage of halls
    for c,t,h in solution:
        h_list.append(h)
    
    h_list.sort()
        
    usage = Counter(h_list)
    
    keys = usage.keys()
    
    for key in keys:
        w = usage[key] - max_hours[key]
        if w > 0:
            fitness += w*5
        
    return fitness

# Function to calculate the initial population for the algorithm
def generate_population(size):
    population = []
    for i in range(size):
        population.append(create_solution())
    return population

# Uses tournament selection for choosing parents
def tournament_select(population, k):
    return [min(random.sample(population, k), key=fitness_function) for i in range(len(population))]

# sort the solution based on the Course number ( i.e   C1,C2,C3,...,Cn )
def sort_p(solution):
    solution.sort(key=lambda x: x[0], reverse=False)

def crossover(parent1, parent2):
    point = random.randint(1, len(courses)-1)
    
    # sort the entries in the parent
    # this step is taken so that we do not produce duplicate values of courses in the offspring tuple
    sort_p(parent1)
    sort_p(parent2)
    
    offspring1 = parent1[:point] + parent2[point:]
    offspring2 = parent2[:point] + parent1[point:]
    return offspring1, offspring2

def mutate(solution):
    course, time, hall = random.choice(solution)
    solution.remove((course, time, hall))
    solution.append((course, random.choice(time_slots), random.choice(exam_halls)))
    return solution

In [4]:
def genetic_algorithm(population_size, generations, tournament_size, crossover_probability, mutation_probability):
    population = generate_population(population_size)
    
    # Running the Algorithm
    for i in range(generations):
        # selecting parents
        parents = tournament_select(population, tournament_size)
        
        # generating offsprings
        offsprings = []
        
        for i in range(0, population_size, 2):
            parent1 = random.choice(parents)
            parent2 = random.choice(parents)
            if random.random() < crossover_probability:
                offspring1, offspring2 = crossover(parent1, parent2)
            else:
                offspring1, offspring2 = parent1, parent2
            if random.random() < mutation_probability:
                offspring1 = mutate(offspring1)
            if random.random() < mutation_probability:
                offspring2 = mutate(offspring2)
            offsprings.append(offspring1)
            offsprings.append(offspring2)

        # Replace the previous population
        population = offsprings

    # Finding best solution (one with the minimum fitness value)
    best_solution = min(population, key=fitness_function)

    return best_solution

In [5]:
population_size = 100
generations = 100
tournament_size = 5
crossover_probability = 0.8
mutation_probability = 0.1


solution = genetic_algorithm(population_size, generations, tournament_size, crossover_probability, mutation_probability)
fitness = fitness_function(solution)

In [6]:
print('Best solution:', solution)
print('Best fitness:', fitness)

Best solution: [('C1', 'T3', 'H2'), ('C2', 'T2', 'H2'), ('C3', 'T2', 'H1'), ('C4', 'T2', 'H2'), ('C5', 'T2', 'H1')]
Best fitness: 0
